# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from collections import Counter
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Код для расчёта энтропии:

In [6]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики точность (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [153]:
import numpy as np
import pandas as pd

class Node:
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.data_left = data_left
        self.data_right = data_right
        self.gain = gain
        self.value = value

class DecisionTree:
  def __init__(self, H, max_depth=5, min_samples_split=5, max_leaves=5):
    self.max_depth = max_depth
    self.min_samples_split = min_samples_split
    self.max_leaves = max_leaves
    self.leaves_num = 0
    self.H = H
    self.root = None
  
  def _information_gain(self, parent, left_c, right_c):
    left_ratio = len(left_c) / len(parent)
    right_ratio = len(right_c) / len(parent)
    return self.H(parent) - (left_ratio * self.H(left_c) + right_ratio * self.H(right_c))
  
  def _best_split(self, X, y):
    best_split = {}
    best_info_gain = -1

    # feature index
    df = pd.concat([X, y], axis=1)

    for feature in X:
      X_curr = X[feature]
      
      for threshold in X_curr:
        df_left = df[df[feature] <= threshold]
        df_right = df[df[feature] > threshold]

        if len(df_left) > 0 and len(df_right) > 0:
          y = df.iloc[:, -1]
          y_left = df_left.iloc[:, -1]
          y_right = df_right.iloc[:, -1]

          gain = self._information_gain(y, y_left, y_right)
          if gain > best_info_gain:
            best_split = {
                'feature': feature,
                'threshold': threshold,
                'df_left': df_left,
                'df_right': df_right,
                'gain': gain
            }
            best_info_gain = gain
    return best_split
  
  def _build(self, X, y, depth=0):
    self.leaves_num -= 1
    n_rows, n_cols = X.shape

    if n_rows >= self.min_samples_split and depth <= self.max_depth and self.leaves_num + 2 <= self.max_leaves:
      self.leaves_num += 2
      best = self._best_split(X, y)

      if best['gain'] > 0:
        left = self._build(
            X = best['df_left'].iloc[:,:-1],
            y = best['df_left'].iloc[:,-1],
            depth = depth + 1
        )
        right = self._build(
            X = best['df_right'].iloc[:,:-1],
            y = best['df_right'].iloc[:,-1],
            depth = depth + 1
        )
        return Node(
            feature = best['feature'],
            threshold = best['threshold'],
            data_left = left,
            data_right = right,
            gain = best['gain']
        )

    # return leaf
    self.leaves_num += 1
    return Node(
        value = Counter(y).most_common(1)[0][0]
    )

  @staticmethod
  def _add_map(a, b):
    for key_b in b:
      if key_b  not in a:
        a[key_b] = 0
      a[key_b] += b[key_b]
    return a

  def _feature_traverse(self, node, features):
    if node.feature == None or node.gain == None:
      return features 

    if node.feature not in features:
      features[node.feature] = 0
    features[node.feature] += node.gain

    if node.data_left != None:
      features = self._add_map(features,
                               self._feature_traverse(node.data_left, features))
    if node.data_right != None:
      features = self._add_map(features,
                               self._feature_traverse(node.data_right, features))
    return features
    

  def feature_importance(self):
    features = {}
    features = self._feature_traverse(self.root, features)
    feat_sum = sum(features.values())
    for f_key in features:
      features[f_key] /= feat_sum

    return features

  
  def fit(self, X, y):
    self.root = self._build(X, y)
  
  def _predict_one(self, x, tree):
    if tree.value != None:
      return tree.value
    feature_value = x[tree.feature]

    if feature_value <= tree.threshold:
      return self._predict_one(x, tree.data_left)

    if feature_value > tree.threshold:
      return self._predict_one(x, tree.data_right)

  def predict(self, X):
    return [self._predict_one(X.iloc[i], self.root) for i in range(len(X))]


In [127]:
from sklearn.datasets import load_iris

iris = load_iris()
data_ir = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
X_iris = data_ir.iloc[:, :-1]
y_iris = data_ir.iloc[:, -1]


In [156]:

dt_1.feature_importance())


{'petal length (cm)': 0.913022262807127,
 'petal width (cm)': 0.07271248001777207,
 'sepal width (cm)': 0.014265257175101088}

In [128]:

X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, random_state=42, test_size=0.2)


In [154]:
dt_1 = DecisionTree(entropy, max_leaves=15)
dt_1.fit(X_train, y_train)
preds_1 = dt_1.predict(X_test)
print(accuracy_score(y_test, preds_1))

dt_2 = DecisionTree(entropy, max_depth=6)
dt_2.fit(X_train, y_train)
preds_2 = dt_2.predict(X_test)
print(accuracy_score(y_test, preds_2))

dt_3 = DecisionTree(entropy, min_samples_split=100)
dt_3.fit(X_train, y_train)
preds_3 = dt_3.predict(X_test)
print(accuracy_score(y_test, preds_3))



1.0
0.9666666666666667
0.6333333333333333


In [62]:
print(type(X_train))

<class 'numpy.ndarray'>


##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [188]:
from sklearn.tree import DecisionTreeClassifier

In [224]:
class RandomForest:
  def __init__(self, random_state=42, trees_amount=20, min_samples_split=10, max_depth=5):
    self.trees_amount = trees_amount
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.random_state = random_state
    np.random.seed(random_state)


    self.decision_trees = []

  @staticmethod
  def _sample_boostrap(X, y):
    n_rows, n_cols = X.shape
    k = round(np.sqrt(n_cols))
    samples = np.random.choice(n_rows, size=n_rows, replace=True)
    
    # deprecated
    # k_samples = np.random.choice(X.columns, size=k, replace=False)
    # return X[k_samples].iloc[samples], y.iloc[samples]

    return X.iloc[samples], y.iloc[samples]

  def fit(self, X, y):
    # refresh given forest if rebuilt
    if len(self.decision_trees) > 0:
      self.decision_trees = []
    num_trees = 0
    while num_trees < self.trees_amount:
      # try:
      dt = DecisionTreeClassifier(criterion="gini",
                        random_state=self.random_state,
                        min_samples_split=self.min_samples_split,
                        max_depth=self.max_depth,
                        max_features="sqrt"
                        )
      X_smpl, y_smpl = self._sample_boostrap(X, y)

      dt.fit(X_smpl, y_smpl)
      self.decision_trees.append(dt)

      num_trees += 1
      # except Exception:
      #   continue

  def predict(self, X):
      y = []
      for tree in self.decision_trees:
        y.append(tree.predict(X))
      
      # so all the predictions to one data row would be in a common list
      y = np.swapaxes(y, 0, 1)
      
      predictions = []
      for preds in y:
        counter = Counter(preds)
        predictions.append(counter.most_common(1)[0][0])
      return predictions
  
  def feature_importance(self, original_score, X_test, y_test):
    predictions = []
    for column in X_test:
      X_cop = X_test.copy()
      X_cop[column] = np.random.permutation(X_cop[column].values)
      pred_score = accuracy_score(y_test, self.predict(X_cop))
      predictions.append(((pred_score - original_score) / original_score, column))
    # print(predictions)
    predictions = sorted(predictions, key=lambda x: x[0])
    return predictions
    
    



    

In [158]:
chur_tbl = pd.read_csv('churn.csv')

In [159]:
chur_tbl = chur_tbl.drop(["Surname", "CustomerId", "Geography"], axis=1)

mapping = {'Female': 1, 'Male' : 0}
chur_tbl = chur_tbl.replace({'Gender': mapping})

X = chur_tbl.iloc[:, 1:-1]
y = chur_tbl["Exited"]


In [160]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [229]:
rf = RandomForest(trees_amount=5, max_depth=6)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
ac_sc = accuracy_score(y_test, pred)


In [226]:
f_imp = rf.feature_importance(ac_sc, X_test, y_test)

In [230]:
pd.DataFrame(f_imp)
# Here you can see how features builded forest depends on a certain features.
# Lower the number –– higher the correlation

,0,1
0,-0.071137,Age
1,-0.043732,NumOfProducts
2,-0.027405,IsActiveMember
3,-0.004665,Balance
4,-0.001749,CreditScore
5,-0.001749,Tenure
6,-0.001166,EstimatedSalary
7,-0.000583,Gender
8,0.000583,HasCrCard


In [223]:
params = {'trees_amount': [11, 13, 15, 20, 30, 40, 60, 80, 125],
           'min_samples_split': [1.0, 2, 3, 4, 7, 8, 10, 13, 15, 18],
           'max_depth': [3, 5, 6, 8, 9, 10]}
best_ac = -1
best_mod = None
for t_a in params['trees_amount']:
  for mss in params['min_samples_split']:
    for md in params['max_depth']:
      rf = RandomForest(trees_amount=t_a, max_depth=md, min_samples_split=mss)
      rf.fit(X_train, y_train)
      pred = rf.predict(X_test)
      ac_sc = accuracy_score(y_test, pred)
      if ac_sc > best_ac:
        best_ac = ac_sc
        best_mod = rf
        best_params = {
            'trees_amount' : t_a,
            'min_samples_split' : mss,
            'max_depth' : md
        }
print(best_ac, best_params)

0.863 {'trees_amount': 30, 'min_samples_split': 18, 'max_depth': 8}
